Немного изучил данные и хочу сделать первую обработку - исправить орфографические ошибки. Для этого я использую "yandexgpt-lite". Был опробован и другой способ - "yandex speller", который предназначен для исправления ошибок в тексте, однако, с длинными предложениями у него возникали проблемы. По этой причине я решил использовать gpt

In [1]:
from tqdm import tqdm
import pandas as pd
import requests
import json

In [2]:
with open("data.json", encoding="UTF-8") as file_in:
    info = json.load(file_in)

text = []
for i in range(len(info)):
    text.append(info[i]["quote"])

In [3]:
text[0]

'«У ннас среди ночi в райооне 55 часов упала полxа с водой в сттеклянной  таре, тоесть, посреди этой лужа, сстеккла, всёё, это уже поод утро, кстати, ээто не перрвыи раз,,, они сами по себе падают, как-то неправилььно рассчитывают, мы же должны выставлять по определённой картинке, у нас при мне было уже пару раз, чтоо сами по себе грохаются эти полки с буттылками» \n\n\n'

По примеру выше видно, что с данными беда. Создаю dataFrame, в котором буду хранить текст без предобратботки и тот, который прошёл через yandexgpt. 

In [4]:
df = pd.DataFrame({"text_original":text, "text_after_preprocess":[None] * len(text)})
df.head()

,text_original,text_after_preprocess
0,«У ннас среди ночi в райооне 55 часов упала по...,None
1,Программы повышения квалификации через гильдию...,None
2,"""Мурр... Новиград - город контрастов, да. Но в...",None
3,"Типа, вот уже полгода, как мы ждем установки э...",None
4,"""Теперь всьё так просто! Раньше бумажки летали...",None


In [2]:
prompt = {
    "modelUri": "gpt://my_key/yandexgpt-lite",
    "completionOptions": {
        "stream": False,
        "temperature": 0.1,
        "maxTokens": "2000"
    },
    "messages": [
        {
            "role": "user",
            "text": ""
        }
    ]
}

url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Api-Key my_key"
}

Ещё раз убедимся, что в данных огромное количество символов. Надо от этого избавляться

In [6]:
def count_unique(data):
    arr_set = set()
    for item in data:
        arr_set.update(item)
    print(arr_set)
    return f"Количество уникальных элементов = {len(arr_set)}"

In [13]:
count_unique(list(df["text_original"]))

{'2', 'ч', '»', '7', 'В', 'п', 'И', 'C', '☕', 'А', '6', 'х', '😜', '🤑', 'E', 'b', 'Щ', '/', 'ц', 'T', 'ë', 'Ж', '1', 'у', 'y', 'z', 'ü', '3', 'щ', 'Е', '😉', ',', '“', 'У', 'n', 'ї', 'ю', '-', 'Ф', 'і', 'Ш', 'с', 'к', 'а', '—', 'ё', 'o', 'ь', 'Н', '�', 'л', 'O', 'm', '"', '5', 'Л', 'я', 'г', 'С', 'Б', 'и', '😡', '$', 'X', '9', 'D', 'q', 'k', '🏡', 'ö', 'ѣ', 'Ю', '.', 'ж', 'B', 'd', '^', 'ó', '€', 'з', 'Р', '💣', 'p', 'б', 'Ы', 'Z', '*', '4', 'й', 'р', '#', 'Д', 'Г', ';', '0', 'è', '🤬', 'é', "'", 'ъ', 'Ё', 'á', '😠', 'т', '&', 'О', '́', '<', 'e', 'v', 'о', 'Й', 'x', 'r', 't', 'Т', 'Э', 'З', '…', 'Х', '«', '+', 'R', 'є', 'ф', 'G', 'u', 'д', 'М', 'е', '\xa0', '(', 'S', 's', 'ы', 'N', 'l', 'м', 'э', 'L', 'a', '🎉', 'в', 'h', 'ı', '”', 'ï', 'I', '😈', 'c', '?', 'ш', 'П', 'M', 'Ч', 'н', 'Я', '!', '@', ':', '8', 'g', 'ò', 'f', 'U', 'i', '%', 'К', 'Ь', 'w', 'Ц', ')', '🔥', '`', ' ', '\n', 'A', 'ł', '\\', 'Q', 'j', 'Y', 'Δ', '–', 'ى', '№', '>', 'K'}


'Количество уникальных элементов = 188'

Сохраняю аббревиатуры в словарь. Аббревиатуры буду заменять на их расшифровки.

In [4]:
abbreviations_dict = {}
with open("abbreviations.txt", encoding="UTF-8") as file:
    for line in file:
        data = line.split(" - ")
        abbreviations_dict[data[0]] = data[1][:-1]
abbreviations_dict

{'СУОЧ': 'Система управления операционными чарами',
 'ЗМС': 'Зачарованная маслостанция',
 'НСС': 'Назаирская система слива',
 'ЛДС': 'Лекарский договор страхования',
 'ЦИСТ': 'Цинтрийский стандарт',
 'СОКГ': 'Система оплаты карточкой гильдии'}

Прохожусь по каждому объекту данных, заменяю аббревиатуры на их расшифровки и прошу yandexgpt вернуть текст без ошибок. Обработанный текст созраняю в DataFrame

In [18]:
text_list = list(df["text_original"])
text_list_new = []

for i in tqdm(range(len(df))):
    if df.loc[i]["text_after_preprocess"] is not None:
        continue
    
    new_text = df.loc[i]["text_original"]
    
    new_text = new_text.strip()
    
    new_text_arr = new_text.split(" ")
    
    for j in range(len(new_text_arr)):
        if new_text_arr[j] in abbreviations_dict:
            new_text_arr[j] = abbreviations_dict[new_text_arr[j]]
           
    new_text = " ".join(new_text_arr)
    
    prompt_text = "Исправь орфографические ошибки в тексте и больше ничего не добавляй в начале и конце текста: "
    
    prompt["messages"][0]["text"] = prompt_text + new_text
    response = requests.post(url, headers=headers, json=prompt)
    result = response.text

    start_index = result.find('"text":') + len('"text":')
    end_index = result.find('"},"status')
    new_text = result[start_index+1:end_index]
   
    df.at[i, "text_after_preprocess"] = new_text

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 959/959 [18:20<00:00,  1.15s/it]


Видно, что текст стал более читабельным

In [9]:
df.head(5)

,text_original,text_after_preprocess
0,«У ннас среди ночi в райооне 55 часов упала по...,У нас среди ночи в районе 55 часов упала полка...
1,Программы повышения квалификации через гильдию...,Программы повышения квалификации через гильдию...
2,"""Мурр... Новиград - город контрастов, да. Но в...","Мурр... Новиград — город контрастов, да. Но в ..."
3,"Типа, вот уже полгода, как мы ждем установки э...","Вот уже полгода, как мы ждём установки этих фи..."
4,"""Теперь всьё так просто! Раньше бумажки летали...","Теперь всё так просто! Раньше бумаги летали, т..."


Сократилось количество уникальных символов. Полное удаление ненужных символов будет произведено в другом ноутбуке

In [8]:
count_unique(list(df["text_after_preprocess"]))

{'c', 'м', 'е', 'ц', '0', '́', 'D', 'у', 'y', ' ', '…', 'b', 'G', '😠', '7', ']', '(', 'к', 'M', 'В', 'Г', '»', '🤑', 'Ч', 'У', '*', '8', '5', 'Э', ',', '😈', 'А', 'J', 'в', 'N', 'd', 'ш', 'я', 'Ф', 's', 'о', 'П', '“', '.', 'ё', '6', 'Б', 'п', 'И', '2', 'т', 'ы', 'r', 'н', 'ф', 'ъ', '🎉', 'а', 'э', 'm', '[', '😡', '💣', 'х', 'Л', 'u', 'o', 'i', 'с', '3', '😉', 'K', 'ж', 'О', '9', 'Р', 'К', 'Е', 'г', 'Ж', 'З', 'С', '–', 'и', 'Ш', 'Q', 'д', 'щ', 'р', ':', 'l', 'n', 'Х', '1', 'P', 'Д', 'A', 'O', 'л', '?', 't', '@', '-', ')', 'б', 'Н', 'T', 'Ц', '—', 'ю', 'ч', '\\', '🔥', 'М', 'ь', '%', "'", 'V', 'p', 'Я', 'e', 'R', '«', '4', '/', 'й', 'h', ';', '!', 'Щ', 'з', 'Т', '„', 'L', 'a'}


'Количество уникальных элементов = 135'

Сохраняю новые объекты и буду продолжать с ними работу в другом ноутбуке.

In [20]:
df.to_csv("data_after_preprocess.csv", index=False)

In [10]:
df_test = pd.read_csv("data_after_preprocess.csv")
df_test.head()

,text_original,text_after_preprocess
0,«У ннас среди ночi в райооне 55 часов упала по...,У нас среди ночи в районе 55 часов упала полка...
1,Программы повышения квалификации через гильдию...,Программы повышения квалификации через гильдию...
2,"""Мурр... Новиград - город контрастов, да. Но в...","Мурр... Новиград — город контрастов, да. Но в ..."
3,"Типа, вот уже полгода, как мы ждем установки э...","Вот уже полгода, как мы ждём установки этих фи..."
4,"""Теперь всьё так просто! Раньше бумажки летали...","Теперь всё так просто! Раньше бумаги летали, т..."
